In [27]:
import os
import json
import time
import requests
import zipfile
from io import BytesIO
from urllib.parse import urlparse
import xml.etree.ElementTree as ET
import pandas as pd
from openai import OpenAI

# 🔑 API 키
DART_API_KEY = "여기 쓰기"
OPENAI_API_KEY = "여기 쓰기"  # 너의 OpenAI 키 입력

client = openai.OpenAI(api_key=OPENAI_API_KEY)

# ✅ 로고 API 키
logo_api_key = "pk_bVmwf0tgQT2enb3JOCA5Ig"

# ✅ 설정
DEFAULT_YEAR = "2023"
DEFAULT_REPRT_CODE = "11011"


## 1차 시도

In [29]:


# ✅ 로고 캐시 경로
LOGO_CACHE_PATH = "summaries/logo_cache.json"
LOGO_API_TOKEN = "여기 쓰기"


# ✅ 캐시 로딩
def load_logo_cache() -> dict:
    if os.path.exists(LOGO_CACHE_PATH):
        with open(LOGO_CACHE_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

# ✅ 캐시 저장
def save_logo_cache(cache: dict):
    os.makedirs(os.path.dirname(LOGO_CACHE_PATH), exist_ok=True)
    with open(LOGO_CACHE_PATH, "w", encoding="utf-8") as f:
        json.dump(cache, f, ensure_ascii=False, indent=2)

# ✅ GPT 기반 slug 추론
def guess_english_slug_from_korean(kor_name: str) -> str:
    prompt = f"""
다음은 한국 기업명입니다: "{kor_name}"
이 기업을 해외에서 소개할 때 가장 많이 쓰이는 영문 기업 이름 또는 도메인용 슬러그를 출력해줘. 단어는 하나 또는 단순한 단어 조합만 사용하고, 소문자로만 출력해. 예를 들어 "삼성전자" → samsung, "농심" → nongshim, "카카오" → kakao 처럼.

오직 슬러그만 출력해.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        slug = response.choices[0].message.content.strip().lower()
        return slug
    except Exception as e:
        print(f"❌ GPT 추론 실패: {e}")
        return ""

# ✅ 로고 URL 생성기 (GPT 추론 + 캐시)
def get_logo_url(kor_company_name: str) -> str:
    logo_cache = load_logo_cache()

    if kor_company_name in logo_cache:
        return logo_cache[kor_company_name]

    slug = guess_english_slug_from_korean(kor_company_name)
    if not slug:
        return "https://img.logo.dev/{slug}.com?token={LOGO_API_TOKEN}&retina=true"

    logo_url = f"https://img.logo.dev/{slug}.com?token={LOGO_API_TOKEN}&retina=true"
    logo_cache[kor_company_name] = logo_url
    save_logo_cache(logo_cache)
    return logo_url

# ✅ 기업 리스트
def load_corp_list(api_key):
    url = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}"
    res = requests.get(url)
    if res.status_code != 200 or b'PK' not in res.content[:2]:
        print("❌ DART에서 corpCode.zip이 유효하지 않게 응답되었습니다.")
        print("🔎 응답 일부 미리보기:\n", res.text[:300])
        raise Exception("❌ corpCode.xml을 zip으로 받아오지 못했습니다.")
    try:
        import xml.etree.ElementTree as ET
        z = zipfile.ZipFile(BytesIO(res.content))
        xml_content = z.read(z.namelist()[0])
        root = ET.fromstring(xml_content)
        return [{"corp_code": c.findtext("corp_code"), "corp_name": c.findtext("corp_name")} for c in root.findall("list")]
    except zipfile.BadZipFile:
        raise Exception("❌ 받은 파일이 zip 형식이 아닙니다.")

# ✅ 기업 개요
def collect_profile(corp_code):
    url = "https://opendart.fss.or.kr/api/company.json"
    params = {"crtfc_key": DART_API_KEY, "corp_code": corp_code}
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return {}
    return {
        "회사명": res.get("corp_name"),
        "대표자": res.get("ceo_nm"),
        "사업자등록번호": res.get("bizr_no"),
        "설립일": res.get("est_dt"),
        "업종코드": res.get("industry_code"),
        "주소": res.get("adres"),
        "지주회사": res.get("hm_ownr_corp_nm"),
        "상장여부": "상장" if res.get("stock_code") else "비상장",
        "홈페이지": res.get("hm_url")
    }

# ✅ 계열사
def collect_affiliates(corp_code):
    url = "https://opendart.fss.or.kr/api/dsclSttus.json"
    params = {
        "crtfc_key": DART_API_KEY,
        "corp_code": corp_code,
        "bsns_year": DEFAULT_YEAR,
        "reprt_code": DEFAULT_REPRT_CODE
    }
    res = requests.get(url, params=params).json()
    if res.get("status") != "000":
        return []
    return [item.get("corp_name") for item in res.get("list", []) if item.get("corp_name")]

# ✅ GPT 요약
def gpt_summary(profile, affiliates):
    categories = "패션, 물류, 핀테크, 유통, 콘텐츠, 플랫폼, 커머스, IT, 미디어, 제조, 기타"
    keywords = "예시: #2030 #남성패션 #MZ, 주요 사업 아이템이나 고객에 대한 정보 제공"
    investor_types = "#공격형, #안정형, #혼합형"

    prompt = f"""당사는 다음과 같은 기업입니다:

[기업 개요]
- 회사명: {profile.get("회사명")}
- 대표자: {profile.get("대표자")}
- 설립일: {profile.get("설립일")}
- 업종코드: {profile.get("업종코드")}
- 주소: {profile.get("주소")}
- 상장여부: {profile.get("상장여부")}

[계열사]
- {', '.join(affiliates) if affiliates else '계열사 정보 없음'}

넌 20년 된 투자 전문가야. 일반인에게 아래 내용을 설명할 거야.

1. 이 기업이 속한 주요 산업 분야를 반드시 다음 중 하나로 판단해서 출력해줘 (그 외는 절대 출력하지 마):
{categories}

2. 다음 중 최대 2개의 키워드를 골라 해시태그로 출력해줘:
{keywords}

3. 반드시 다음 중 하나를 세 번째 키워드로 붙여줘 (항상 마지막에 위치해야 해):
{investor_types}

4. 이 기업을 전문가 관점에서 핵심 요점만 뽑아 한 문장으로 요약해줘.

출력 형식은 다음과 같아야 해:

{{
  "주요 분야": "패션",
  "키워드": ["#2030", "#남성패션", "#혼합형"],
  "한 문장 요약": "이 기업은 ..."
}}"""
    try:
        res = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5
        )
        return json.loads(res.choices[0].message.content.strip())
    except Exception as e:
        return {
            "주요 분야": "정보 없음",
            "키워드": ["#정보없음"],
            "한 문장 요약": f"❌ GPT 요약 실패: {e}"
        }

# ✅ 실행 함수
def run_batch_for_keyword(keyword):
    corp_list = load_corp_list(DART_API_KEY)
    keyword_lower = keyword.lower()
    matches = [c for c in corp_list if keyword_lower in c["corp_name"].lower()]
    matches.sort(key=lambda c: (not c["corp_name"].startswith(keyword), c["corp_name"]))
    matches = matches[:20]

    print(f"\n✅ '{keyword}'가 포함된 기업 {len(matches)}개 결과:\n")

    for corp in matches:
        print(f"🔍 {corp['corp_name']} (corp_code: {corp['corp_code']})")
        profile = collect_profile(corp["corp_code"])
        affiliates = collect_affiliates(corp["corp_code"])
        logo_url = get_logo_url(profile.get("홈페이지", ""))
        summary = gpt_summary(profile, affiliates)

        result_dict = {
            "회사명": profile.get("회사명"),
            "대표자": profile.get("대표자"),
            "상장여부": profile.get("상장여부"),
            "설립일": profile.get("설립일"),
            "주소": profile.get("주소"),
            "사업자등록번호": profile.get("사업자등록번호"),
            "로고": logo_url,
            "주요 분야": summary.get("주요 분야"),
            "키워드": summary.get("키워드"),
            "GPT 요약": summary.get("한 문장 요약")
        }

        if affiliates:
            result_dict["계열사"] = affiliates

        print(json.dumps(result_dict, indent=2, ensure_ascii=False))
        print("=" * 100)
        time.sleep(1.2)

# ✅ 진입점
if __name__ == "__main__":
    keyword = input("🔍 검색할 기업 키워드를 입력하세요: ")
    run_batch_for_keyword(keyword)


KeyboardInterrupt: Interrupted by user